[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/abigailhaddad/fedscope_new/blob/main/demo.ipynb)

# OPM Federal Workforce Data: Accessions & Separations Over Time

This notebook loads federal workforce accession (new hires) and separation (departures) data from HuggingFace and visualizes trends over time.

**No authentication required** - all datasets are public.

In [ ]:
# Install dependencies (for Colab)
!pip install -q duckdb pandas plotly

In [ ]:
import duckdb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

## 1. Define Dataset URLs

The data is stored as parquet files on HuggingFace. DuckDB can query them directly via HTTP.

In [ ]:
# Generate all dataset URLs
HF_USERNAME = "abigailhaddad"
BASE_URL = f"https://huggingface.co/datasets/{HF_USERNAME}"

def generate_months(start_year=2021, start_month=1, end_year=2025, end_month=11):
    """Generate list of YYYYMM strings."""
    months = []
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            if year == start_year and month < start_month:
                continue
            if year == end_year and month > end_month:
                break
            months.append(f"{year}{month:02d}")
    return months

months = generate_months()

# Build URLs for each dataset
accession_urls = [f"{BASE_URL}/opm-federal-accessions-{m}/resolve/main/data.parquet" for m in months]
separation_urls = [f"{BASE_URL}/opm-federal-separations-{m}/resolve/main/data.parquet" for m in months]

print(f"Generated {len(accession_urls)} accession URLs")
print(f"Generated {len(separation_urls)} separation URLs")
print(f"\nExample: {accession_urls[0]}")

In [ ]:
# Verify datasets exist by checking a few
def check_url(url):
    """Quick check if parquet URL is accessible."""
    try:
        duckdb.sql(f"SELECT 1 FROM '{url}' LIMIT 1")
        return True
    except:
        return False

# Check first and last of each type
print("Checking dataset availability...")
print(f"  First accession ({months[0]}): {'OK' if check_url(accession_urls[0]) else 'MISSING'}")
print(f"  Last accession ({months[-1]}): {'OK' if check_url(accession_urls[-1]) else 'MISSING'}")
print(f"  First separation ({months[0]}): {'OK' if check_url(separation_urls[0]) else 'MISSING'}")
print(f"  Last separation ({months[-1]}): {'OK' if check_url(separation_urls[-1]) else 'MISSING'}")

## 2. Load All Data with DuckDB

DuckDB reads parquet files directly via HTTP - no download needed. This is much faster than the datasets library.

In [ ]:
%%time
from concurrent.futures import ThreadPoolExecutor, as_completed

def get_count(args):
    """Get row count for a single parquet URL."""
    url, month = args
    try:
        # Each thread needs its own connection
        conn = duckdb.connect()
        result = conn.sql(f"SELECT COUNT(*) FROM '{url}'").fetchone()
        conn.close()
        return {'month': month, 'date': datetime.strptime(month, '%Y%m'), 'count': result[0]}
    except:
        return None

def load_counts_parallel(urls, months, data_type, max_workers=16):
    """Load counts in parallel using thread pool."""
    counts = []
    errors = []
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(get_count, (url, month)): month 
                   for url, month in zip(urls, months)}
        
        for future in as_completed(futures):
            result = future.result()
            if result:
                counts.append(result)
            else:
                errors.append(futures[future])
    
    if errors:
        print(f"  Missing {data_type} months: {sorted(errors)}")
    else:
        print(f"  All {len(counts)} {data_type} months loaded!")
    
    # Sort by month
    return pd.DataFrame(counts).sort_values('month').reset_index(drop=True)

print("Loading accessions and separations in parallel...")
acc_df = load_counts_parallel(accession_urls, months, "accession")
sep_df = load_counts_parallel(separation_urls, months, "separation")

print(f"\nLoaded {len(acc_df)} accession months, {len(sep_df)} separation months")

## 3. Plot Accessions vs Separations Over Time

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=acc_df['date'], 
    y=acc_df['count'],
    mode='lines+markers',
    name='Accessions (New Hires)',
    line=dict(color='#2ecc71', width=2),
    marker=dict(size=6),
    hovertemplate='%{x|%b %Y}<br>%{y:,.0f} new hires<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=sep_df['date'], 
    y=sep_df['count'],
    mode='lines+markers',
    name='Separations (Departures)',
    line=dict(color='#e74c3c', width=2),
    marker=dict(size=6),
    hovertemplate='%{x|%b %Y}<br>%{y:,.0f} departures<extra></extra>'
))

fig.update_layout(
    title='Federal Workforce: Monthly Accessions vs Separations (2021-2025)',
    xaxis_title='Month',
    yaxis_title='Count',
    hovermode='x unified',
    template='plotly_white',
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    ),
    yaxis=dict(tickformat=','),
    height=500
)

fig.show()

In [ ]:
# Net change (accessions - separations)
merged = acc_df.merge(sep_df, on=['month', 'date'], suffixes=('_acc', '_sep'))
merged['net_change'] = merged['count_acc'] - merged['count_sep']
merged['color'] = merged['net_change'].apply(lambda x: '#2ecc71' if x > 0 else '#e74c3c')

fig = go.Figure()

fig.add_trace(go.Bar(
    x=merged['date'],
    y=merged['net_change'],
    marker_color=merged['color'],
    hovertemplate='%{x|%b %Y}<br>Net: %{y:+,.0f}<extra></extra>'
))

fig.add_hline(y=0, line_width=1, line_color='black')

fig.update_layout(
    title='Federal Workforce: Monthly Net Change (Accessions - Separations)',
    xaxis_title='Month',
    yaxis_title='Net Change',
    template='plotly_white',
    yaxis=dict(tickformat=','),
    height=450,
    showlegend=False
)

fig.show()

print(f"\nTotal accessions: {merged['count_acc'].sum():,}")
print(f"Total separations: {merged['count_sep'].sum():,}")
print(f"Net change: {merged['net_change'].sum():+,}")

## 4. Explore the Data Structure

Let's look at what columns are available in the data.

In [ ]:
# Inspect columns using DuckDB
print("Accessions columns:")
acc_cols = duckdb.sql(f"DESCRIBE SELECT * FROM '{accession_urls[0]}'").df()
print(acc_cols['column_name'].tolist())

print("\nSeparations columns:")
sep_cols = duckdb.sql(f"DESCRIBE SELECT * FROM '{separation_urls[0]}'").df()
print(sep_cols['column_name'].tolist())

In [ ]:
# Show sample data
print("Sample accession records:")
duckdb.sql(f"SELECT * FROM '{accession_urls[0]}' LIMIT 5").df()

## 5. Data Quality Analysis

Some fields have missing, redacted, or placeholder values that can affect analysis.

In [ ]:
# Analyze data quality across all accession data
# Using just the most recent month for speed - patterns are consistent

sample_url = accession_urls[-1]  # Most recent month

quality_issues = []

# Check key columns for problematic values
checks = {
    'duty_station_state': ['REDACTED', 'NO DATA REPORTED', 'UNSPECIFIED'],
    'education_level': ['UNSPECIFIED', 'NO DATA REPORTED'],
    'occupational_series': ['UNSPECIFIED', 'NO DATA REPORTED'],
    'agency': ['UNSPECIFIED'],
    'stem_occupation': ['UNSPECIFIED', 'ALL OTHER OCCUPATIONS'],
    'supervisory_status': ['ALL OTHER POSITIONS'],
}

print("=== DATA QUALITY ISSUES (sample month) ===\n")

for col, bad_values in checks.items():
    conditions = " OR ".join([f"{col} = '{v}'" for v in bad_values])
    
    result = duckdb.sql(f"""
        SELECT 
            {col} as value,
            SUM(CAST(count AS INTEGER)) as people
        FROM '{sample_url}'
        WHERE {conditions}
        GROUP BY {col}
        ORDER BY people DESC
    """).df()
    
    if len(result) > 0:
        total = result['people'].sum()
        total_all = duckdb.sql(f"SELECT SUM(CAST(count AS INTEGER)) FROM '{sample_url}'").fetchone()[0]
        pct = (total / total_all) * 100
        
        print(f"📍 {col}: {total:,.0f} people ({pct:.1f}%) have unusable values")
        for _, row in result.iterrows():
            print(f"   • '{row['value']}': {row['people']:,.0f}")
        print()

## 6. Interactive Filtering

Filter accessions and separations by agency, state, occupation, etc. and view the trends.

In [ ]:
# Get available filter options from a sample month
sample_url = accession_urls[-1]

def get_options(col):
    """Get unique values for a column, sorted by frequency."""
    result = duckdb.sql(f"""
        SELECT {col} as value, SUM(CAST(count AS INTEGER)) as n
        FROM '{sample_url}'
        WHERE {col} IS NOT NULL AND {col} != ''
        GROUP BY {col}
        ORDER BY n DESC
    """).df()
    return ['All'] + result['value'].tolist()

# Cache filter options
filter_options = {
    'agency': get_options('agency'),
    'duty_station_state': get_options('duty_station_state'),
    'occupational_group': get_options('occupational_group'),
    'age_bracket': get_options('age_bracket'),
    'education_level': get_options('education_level'),
}

print("Filter options loaded!")
print(f"  Agencies: {len(filter_options['agency'])-1}")
print(f"  States: {len(filter_options['duty_station_state'])-1}")
print(f"  Occupational Groups: {len(filter_options['occupational_group'])-1}")
print(f"  Age Brackets: {len(filter_options['age_bracket'])-1}")
print(f"  Education Levels: {len(filter_options['education_level'])-1}")

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create filter widgets
agency_dropdown = widgets.Dropdown(options=filter_options['agency'], value='All', description='Agency:')
state_dropdown = widgets.Dropdown(options=filter_options['duty_station_state'], value='All', description='State:')
occ_dropdown = widgets.Dropdown(options=filter_options['occupational_group'], value='All', description='Occ Group:')

# Output area for results
output = widgets.Output()
download_output = widgets.Output()

def query_filtered_data(data_type, agency, state, occ_group):
    """Query all months with filters applied."""
    urls = accession_urls if data_type == 'accessions' else separation_urls
    
    # Build WHERE clause
    conditions = []
    if agency != 'All':
        conditions.append(f"agency = '{agency}'")
    if state != 'All':
        conditions.append(f"duty_station_state = '{state}'")
    if occ_group != 'All':
        conditions.append(f"occupational_group = '{occ_group}'")
    
    where_clause = "WHERE " + " AND ".join(conditions) if conditions else ""
    
    results = []
    for url, month in zip(urls, months):
        try:
            conn = duckdb.connect()
            result = conn.sql(f"""
                SELECT SUM(CAST(count AS INTEGER)) as total
                FROM '{url}'
                {where_clause}
            """).fetchone()
            conn.close()
            if result[0]:
                results.append({
                    'month': month,
                    'date': datetime.strptime(month, '%Y%m'),
                    'count': result[0]
                })
        except:
            pass
    
    return pd.DataFrame(results)

def update_chart(change=None):
    """Update the chart based on filter selections."""
    with output:
        clear_output(wait=True)
        print("Loading filtered data...")
        
        # Query both datasets in parallel
        from concurrent.futures import ThreadPoolExecutor
        with ThreadPoolExecutor(max_workers=2) as executor:
            acc_future = executor.submit(
                query_filtered_data, 'accessions', 
                agency_dropdown.value, state_dropdown.value, occ_dropdown.value
            )
            sep_future = executor.submit(
                query_filtered_data, 'separations',
                agency_dropdown.value, state_dropdown.value, occ_dropdown.value
            )
            filtered_acc = acc_future.result()
            filtered_sep = sep_future.result()
        
        clear_output(wait=True)
        
        if len(filtered_acc) == 0 and len(filtered_sep) == 0:
            print("No data found for these filters.")
            return
        
        # Create chart
        fig = go.Figure()
        
        if len(filtered_acc) > 0:
            fig.add_trace(go.Scatter(
                x=filtered_acc['date'], y=filtered_acc['count'],
                mode='lines+markers', name='Accessions',
                line=dict(color='#2ecc71', width=2),
                hovertemplate='%{x|%b %Y}<br>%{y:,.0f}<extra></extra>'
            ))
        
        if len(filtered_sep) > 0:
            fig.add_trace(go.Scatter(
                x=filtered_sep['date'], y=filtered_sep['count'],
                mode='lines+markers', name='Separations',
                line=dict(color='#e74c3c', width=2),
                hovertemplate='%{x|%b %Y}<br>%{y:,.0f}<extra></extra>'
            ))
        
        # Build title
        filters = []
        if agency_dropdown.value != 'All':
            filters.append(agency_dropdown.value[:30])
        if state_dropdown.value != 'All':
            filters.append(state_dropdown.value)
        if occ_dropdown.value != 'All':
            filters.append(occ_dropdown.value[:30])
        
        title = "Filtered: " + ", ".join(filters) if filters else "All Federal Workforce"
        
        fig.update_layout(
            title=title,
            xaxis_title='Month', yaxis_title='Count',
            template='plotly_white', height=450,
            yaxis=dict(tickformat=','),
            hovermode='x unified'
        )
        fig.show()
        
        # Show totals
        acc_total = filtered_acc['count'].sum() if len(filtered_acc) > 0 else 0
        sep_total = filtered_sep['count'].sum() if len(filtered_sep) > 0 else 0
        print(f"\nTotal accessions: {acc_total:,.0f}")
        print(f"Total separations: {sep_total:,.0f}")
        print(f"Net change: {acc_total - sep_total:+,.0f}")
    
    # Update download data
    with download_output:
        clear_output(wait=True)
        if len(filtered_acc) > 0 or len(filtered_sep) > 0:
            merged = pd.merge(
                filtered_acc.rename(columns={'count': 'accessions'}),
                filtered_sep.rename(columns={'count': 'separations'}),
                on=['month', 'date'], how='outer'
            ).fillna(0)
            merged['net_change'] = merged['accessions'] - merged['separations']
            print("📥 Data ready for download (right-click table → Save as CSV):")
            display(merged[['month', 'accessions', 'separations', 'net_change']])

# Connect widgets to update function
agency_dropdown.observe(update_chart, names='value')
state_dropdown.observe(update_chart, names='value')
occ_dropdown.observe(update_chart, names='value')

# Display widgets
print("Select filters and the chart will update automatically:")
display(widgets.HBox([agency_dropdown, state_dropdown, occ_dropdown]))
display(output)
display(download_output)

# Initial load
update_chart()